In [1]:
spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1619393293929_0006,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 3.0.1-amzn-0


In [2]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sagemaker-coherehealth-1/spark/deequ-1.2.1-spark-3.0.jar"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1619393293929_0007,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1619393293929_0007,spark,idle,Link,Link,✔


In [3]:
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.profiles.{ColumnProfilerRunner, NumericColumnProfile, ColumnProfile, ColumnProfiles}
import com.amazon.deequ.analyzers.{DataTypeInstances}
import com.amazon.deequ.analyzers.runners.{AnalysisRunner, AnalyzerContext}
import com.amazon.deequ.analyzers.runners.AnalyzerContext.successMetricsAsDataFrame
import com.amazon.deequ.analyzers.{Compliance, Correlation, Size, Completeness, Mean, ApproxCountDistinct}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.checks.{Check, CheckLevel, CheckStatus}
import com.amazon.deequ.profiles.{ColumnProfilerRunner, NumericColumnProfile, ColumnProfile, ColumnProfiles}
import com.amazon.deequ.analyzers.DataTypeInstances
import com.amazon.deequ.analyzers.runners.{AnalysisRunner, AnalyzerContext}
import com.amazon.deequ.analyzers.runners.AnalyzerContext.successMetricsAsDataFrame
import com.amazon.deequ.analyzers.{Compliance, Correlation, Size, Completeness, Mean, ApproxCountDistinct}
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


In [4]:
def withTimeChecker(fn:()=> Any) = {
    val t1 = System.nanoTime
    val ret = fn()
    println(s"Operation took ${(System.nanoTime - t1)/1e9d} seconds")
    ret
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

withTimeChecker: (fn: () => Any)Any


In [5]:
val df_claim = withTimeChecker { () => 
    val name = "HUMANA_MEDCLAIM_201811.txt"
    val input_path = "s3a://sagemaker-coherehealth-1/"
    val df = spark.read.option("delimiter", "|").option("header", true).csv(input_path + name)
    df.printSchema()
    df
}.asInstanceOf[DataFrame]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SOURCE PLATFORM CD: string (nullable = true)
 |-- SOURCE CLAIM NBR: string (nullable = true)
 |-- SOURCE CLAIM SUFFIX: string (nullable = true)
 |-- SOURCE LINE ID: string (nullable = true)
 |-- SOURCE LINE SUFFIX: string (nullable = true)
 |-- PYMT CATEGORY CD: string (nullable = true)
 |-- DENIED EXPLANATION CD: string (nullable = true)
 |-- SOURCE MEMBER ID: string (nullable = true)
 |-- MEMBER KEY: string (nullable = true)
 |-- MEMBER AGE: string (nullable = true)
 |-- SOURCE AUTHORIZATION KEY: string (nullable = true)
 |-- PLAN GEN KEY: string (nullable = true)
 |-- CLAIM FORM TYPE CD: string (nullable = true)
 |-- PAID DATE SKEY: string (nullable = true)
 |-- SERVICE FROM DATE SKEY: string (nullable = true)
 |-- SERVICE TO DATE SKEY: string (nullable = true)
 |-- CLAIM RECEIPT DATE SKEY: string (nullable = true)
 |-- BILL TYPE CD: string (nullable = true)
 |-- PLACE OF TREATMENT CD: string (nullable = true)
 |-- SOURCE ADMIT DATE SKEY: string (nullable = true)
 |-- ADMI

In [6]:
val result = withTimeChecker {() => 
    ColumnProfilerRunner().
      onData(df_claim).
      run()
}.asInstanceOf[ColumnProfiles]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Operation took 300.827214838 seconds
result: com.amazon.deequ.profiles.ColumnProfiles = ColumnProfiles(Map(ICD SOURCE VERSION CD -> NumericColumnProfile(ICD SOURCE VERSION CD,1.0,1,Integral,true,Map(Boolean -> 0, Fractional -> 0, Integral -> 10367753, Unknown -> 0, String -> 0),Some(Distribution(Map(10 -> DistributionValue(10367753,1.0)),1)),None,Some(10.0),Some(10.0),Some(10.0),Some(1.0367753E8),Some(0.0),None), ICD PROCEDURE 09 CD -> StandardColumnProfile(ICD PROCEDURE 09 CD,6.248219840885484E-4,108,String,true,Map(Boolean -> 0, Fractional -> 40, Integral -> 0, Unknown -> 10361275, String -> 6438),Some(Distribution(Map(30233N1 -> DistributionValue(417,4.02208656012542E-5), 5A1221Z -> DistributionValue(40,3.858116604436853E-6), 0DU407Z -> DistributionValue(1,9.645291511092134E-8), 0CDWXZ2 -> DistributionValue(99,9.5488385...


In [7]:
object Category extends Enumeration {
    type Category = Value
    val full = Value("full")
    val high = Value("High")
    val med = Value("Medium")
    val low = Value("Low")
    val rare= Value("Rare")
}

case class Profile(column: String, profile: ColumnProfile){
    def getCategory(): Category.Value = {
        this.profile.completeness match {
            case x if x > 0.999 => Category.full
            case x if x > 0.85 => Category.high
            case x if x > 0.35 => Category.med
            case x if x > 0.01 => Category.low
            case _ => Category.rare
        }
    }
}

case class Coverage(full: List[Profile], high: List[Profile], med: List[Profile], low: List[Profile], rare:List[Profile]) {
    def this() = this(List[Profile](), List[Profile](), List[Profile](), List[Profile](), List[Profile]())
    def +(other: Coverage) = Coverage(this.full ++ other.full, this.high ++ other.high, this.med ++ other.med, this.low ++ other.low, this.rare ++ other.rare)
}

object Coverage {
    def byCategory(item: Profile) = {
        item.getCategory match {
            case Category.full => new Coverage(List[Profile](item), List[Profile](), List[Profile](), List[Profile](), List[Profile]())
            case Category.high => new Coverage(List[Profile](), List[Profile](item), List[Profile](), List[Profile](), List[Profile]())
            case Category.med => new Coverage(List[Profile](), List[Profile](), List[Profile](item), List[Profile](), List[Profile]())
            case Category.low => new Coverage(List[Profile](), List[Profile](), List[Profile](), List[Profile](item), List[Profile]())
            case Category.rare => new Coverage(List[Profile](), List[Profile](), List[Profile](), List[Profile](), List[Profile](item))
        }
    }
}

def Categorize(items: Iterable[Profile]): Coverage = {
    if(items.isEmpty) {
        return new Coverage()
    }
    val head::tail = items
    Coverage.byCategory(head)+ Categorize(tail)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Category
defined class Profile
defined class Coverage
defined object Coverage
Companions must be defined together; you may wish to use :paste mode for this.
Categorize: (items: Iterable[Profile])Coverage


In [8]:
val Coverages = Categorize(result.profiles.map( e => Profile(e._1, e._2)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coverages: Coverage = Coverage(List(Profile(ICD SOURCE VERSION CD,NumericColumnProfile(ICD SOURCE VERSION CD,1.0,1,Integral,true,Map(Boolean -> 0, Fractional -> 0, Integral -> 10367753, Unknown -> 0, String -> 0),Some(Distribution(Map(10 -> DistributionValue(10367753,1.0)),1)),None,Some(10.0),Some(10.0),Some(10.0),Some(1.0367753E8),Some(0.0),None)), Profile(SOURCE MEMBER ID,NumericColumnProfile(SOURCE MEMBER ID,1.0,1068130,Integral,true,Map(Boolean -> 0, Fractional -> 0, Integral -> 10367753, Unknown -> 0, String -> 0),None,None,Some(1.9814943674434142E9),Some(9.9997176201E10),Some(14.0),Some(2.054364417254456E16),Some(7.028537075214638E9),None)), Profile(LOGICAL CLAIM RESTATEMENT FINANCIAL CATEGORY CD,StandardColumnProfile(LOGICAL CLAIM RESTATEMENT FINANCIAL CATEGORY CD,1.0,16,String,t...


In [9]:
println("/n ************* Full Coverage Columns *******************")
Coverages.full.map(p => println (s"${p.column}: ${p.profile.approximateNumDistinctValues} Distinct Values"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/n ************* Full Coverage Columns *******************
ICD SOURCE VERSION CD: 1 Distinct Values
SOURCE MEMBER ID: 1068130 Distinct Values
LOGICAL CLAIM RESTATEMENT FINANCIAL CATEGORY CD: 16 Distinct Values
PLAN GEN KEY: 5681 Distinct Values
COB PAID AMT: 9631 Distinct Values
SERVICE TO DATE SKEY: 121 Distinct Values
PRIMARY DIAG CD: 20716 Distinct Values
MEMBER KEY: 983795 Distinct Values
SERVICE UNIT CNT: 5572 Distinct Values
SOURCE CLAIM NBR: 3668963 Distinct Values
PYMT REDUCTION AMT: 43860 Distinct Values
LOGICAL CLAIM RESTATEMENT TREND CATEGORY DESCRIPTION: 7 Distinct Values
PAID DATE SKEY: 697 Distinct Values
SOURCE LINE ID: 316 Distinct Values
EXCL DAYS CNT: 53 Distinct Values
LOGICAL CLAIM RESTATEMENT FINANCIAL CATEGORY DESCRIPTION: 16 Distinct Values
SOURCE NETWORK ID: 219 Distinct Values
CHARGE AMT: 281515 Distinct Values
PLACE OF TREATMENT CD: 44 Distinct Values
DIAG LINE POINTER NBR: 10 Distinct Values
SOURCE LINE SUFFIX: 5004489 Distinct Values
COPAY AMT: 21185 Distinc

In [11]:
Coverages.full.filter(p => p.column == "MEMBER KEY").map(p => p.profile.asInstanceOf[NumericColumnProfile]).foreach { p =>
    println("**** Member Age ****")
    println(s"\t**Statistics\n" +
      s"\tminimum: ${p.minimum.get}\n" +
      s"\tmaximum: ${p.maximum.get}\n" +
      s"\tmean: ${p.mean.get}\n" +
      s"\tstandard deviation: ${p.stdDev.get}\n"
    )
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**** Member Age ****
	**Statistics
	minimum: 1.0
	maximum: 4.5570039643E10
	mean: 3.611826771526086E10
	standard deviation: 4.360883291049699E9



In [19]:
df_claim.filter($"MEMBER KEY" === "1").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+--------------------+--------------+------------------+----------------+---------------------+----------------+----------+----------+------------------------+-------------+------------------+--------------+----------------------+--------------------+-----------------------+------------+---------------------+----------------------+-------------+---------------+--------------------------+-------------------+---------------------------+----------------------------+--------------------------+---------------------------+-----------------------------+-----------------------------------+-----------------------------+-----------------------------------+------------------------------------+------------------------------------------+-------------------------------+-------------------------------------+---------------------------------------+---------------------------------------------+---------------+-----------------+-----------------+---------------+------

In [30]:
println("/n ************* high Coverage Columns *******************")
Coverages.high.map(p => println (s"${p.column}: ${p.profile.completeness} ${p.profile.approximateNumDistinctValues} Distinct Values"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/n ************* high Coverage Columns *******************
OPERATING PHYSICIAN SOURCE PROVIDER KEY: 0.9703565468814699 47259 Distinct Values
RENDERING SOURCE PROVIDER KEY: 0.9703565468814699 285830 Distinct Values
BILLING NATIONAL PROVIDER ID: 0.9900237785371623 267692 Distinct Values
DENIED EXPLANATION CD: 0.8534916871572847 155 Distinct Values
SOURCE CLAIM SUFFIX: 0.8534096057265254 793751 Distinct Values
HCPCS CPT4 TYPE CD 1: 0.8766768459858177 3 Distinct Values
PAY TO NATIONAL PROVIDER ID: 0.9900237785371623 267692 Distinct Values
REFERRING SOURCE PROVIDER 1 KEY: 0.9703565468814699 371699 Distinct Values
HCPCS CPT4 BASE CD 1: 0.8766768459858177 10002 Distinct Values
ATTENDING SOURCE PROVIDER KEY: 0.9703565468814699 160972 Distinct Values
SERVICE FACILITY SOURCE PROVIDER KEY: 0.9703565468814699 176068 Distinct Values
res14: List[Unit] = List((), (), (), (), (), (), (), (), (), (), ())


In [31]:
println("/n ************* medium Coverage Columns *******************")
Coverages.med.map(p => println (s"${p.column}: ${p.profile.completeness} ${p.profile.approximateNumDistinctValues} Distinct Values"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/n ************* medium Coverage Columns *******************
RAW CLAIM ATTENDING PROVIDER NPI ID: 0.4366752371511937 123438 Distinct Values
RAW CLAIM RENDERING PROVIDER NPI ID: 0.4438814273449609 273235 Distinct Values
DIAG CD 3: 0.6296272683193745 16234 Distinct Values
HCPCS CPT4 SUFFIX CD 1: 0.46228020671402953 35 Distinct Values
ADMIT TYPE CD: 0.4090598994787009 7 Distinct Values
BILL TYPE CD: 0.4095412959780195 128 Distinct Values
RAW CLAIM REFERRING PROVIDER 1 NPI ID: 0.4417292734500909 219789 Distinct Values
DIAG CD 2: 0.7829625184936408 19124 Distinct Values
DISCHARGE STATUS CD: 0.7876170468181486 44 Distinct Values
DIAG CD 4: 0.5165219503203828 13552 Distinct Values
CONTRACT METHOD OF PYMT CD: 0.5973345429814927 12 Distinct Values
ADMIT SOURCE CD: 0.4090586455908045 14 Distinct Values
DIAG CD 5: 0.4061389193974818 10578 Distinct Values
REVENUE CD: 0.43731520224295467 382 Distinct Values
res16: List[Unit] = List((), (), (), (), (), (), (), (), (), (), (), (), (), ())


In [32]:
println("/n ************* low Coverage Columns *******************")
Coverages.low.map(p => println (s"${p.column}: ${p.profile.completeness} ${p.profile.approximateNumDistinctValues} Distinct Values"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/n ************* low Coverage Columns *******************
CPT MODIFIER CD: 0.2641392016187114 355 Distinct Values
BEHAVIORAL HEALTH VENDOR CD: 0.013229192477868638 1 Distinct Values
ICD PROCEDURE 06 CD: 0.010786908214344998 879 Distinct Values
ICD PROCEDURE 05 CD: 0.014679265603646229 1146 Distinct Values
DIAG CD 6: 0.34317783226510123 9468 Distinct Values
SOURCE CONTRACT PAR CD: 0.14658817585642714 7 Distinct Values
DIAG CD 4 POA CD: 0.07767150702760763 5 Distinct Values
DIAG CD 11: 0.02247478310874111 1979 Distinct Values
DIAG CD 15: 0.01253656409445711 1092 Distinct Values
ICD PROCEDURE 04 CD: 0.020272666603843668 1348 Distinct Values
DIAG CD 3 POA CD: 0.07908782163309638 4 Distinct Values
RAW CLAIM SERVICE FACILITY PROVIDER NPI ID: 0.2638034490212103 41834 Distinct Values
CAUSE SUB CD: 0.11391696927965009 231 Distinct Values
DIAG CD 7: 0.28529479820748044 8387 Distinct Values
DIAG CD 6 POA CD: 0.07313484416536543 4 Distinct Values
DIAG CD 10: 0.026011615052943488 2430 Distinct Valu

In [44]:
println("/n ************* rare Coverage Columns *******************")
Coverages.rare.map(p => println (s"${p.column}: ${p.profile.completeness} ${p.profile.approximateNumDistinctValues} Distinct Values"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/n ************* rare Coverage Columns *******************
ICD PROCEDURE 09 CD: 6.248219840885484E-4 108 Distinct Values
ICD PROCEDURE 14 CD: 1.7197554764277273E-4 42 Distinct Values
DIAG CD 19 POA CD: 0.001557039408635603 4 Distinct Values
DIAG CD 22 POA CD: 9.936579314727117E-4 4 Distinct Values
ICD PROCEDURE 13 CD: 1.894335252778495E-4 47 Distinct Values
DIAG CD 10 POA CD: 0.004992306433226177 4 Distinct Values
ICD PROCEDURE 16 CD: 1.3541989281573355E-4 44 Distinct Values
ICD PROCEDURE 24 CD: 4.4561246781245655E-5 12 Distinct Values
DIAG CD 22: 0.0044975029787071505 518 Distinct Values
CPT MODIFIER CD 3: 0.009394321025973516 182 Distinct Values
DIAG CD 15 POA CD: 0.0023879812723161904 4 Distinct Values
CPT MODIFIER CD 4: 2.7913473633100637E-4 79 Distinct Values
ICD PROCEDURE 18 CD: 1.1950516182243153E-4 31 Distinct Values
DIAG CD 18 POA CD: 0.0017392389652801335 4 Distinct Values
DIAG CD 13 POA CD: 0.0029922587854861126 4 Distinct Values
ICD PROCEDURE 25 CD: 4.4464793866134735E-5 11

In [34]:
df_claim.select("SOURCE CLAIM NBR", "SOURCE CLAIM SUFFIX").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+
|SOURCE CLAIM NBR| SOURCE CLAIM SUFFIX|
+----------------+--------------------+
| 820190700130157|584025025264SHIRL...|
| 820190710438204|572551333292ROSA ...|
| 820190740972652|582352901921ELIZA...|
| 820190740972652|582352901921ELIZA...|
| 820190740972652|582352901921ELIZA...|
| 820183260219012|582964062474MARY ...|
| 820183260219012|582964062474MARY ...|
| 820183310262184|584047861372FRANC...|
| 820183310262184|584047861372FRANC...|
| 820183310262184|584047861372FRANC...|
| 820183310262184|584047861372FRANC...|
| 820190670016733|584057485056MICHA...|
| 820183520351971|584128847771JUDY ...|
| 820183130139819|581754258972ROSEM...|
| 820183470975768|584906069314RICHA...|
| 820183510200851|572526241832LOIS ...|
| 820183510200851|572526241832LOIS ...|
| 820183520398946|57366806108 STEVE...|
| 820183530240401|583836096673DIANE...|
| 820183530240401|583836096673DIANE...|
+----------------+--------------------+
only showing top 20 rows



In [35]:
def getStatistics(cov:List[Profile], column:String) = {
    cov.filter(p => p.column == column).map(p => p.profile.asInstanceOf[NumericColumnProfile]).foreach { p =>
            println(s"**** $column ****")
            println(s"\t**Statistics\n" +
              s"\tminimum: ${p.minimum.get}\n" +
              s"\tmaximum: ${p.maximum.get}\n" +
              s"\tmean: ${p.mean.get}\n" +
              s"\tstandard deviation: ${p.stdDev.get}\n"
        )
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getStatistics: (cov: List[Profile], column: String)Unit


In [36]:
def getHistogram(cov:List[Profile], column:String) = {
    cov.filter(p => p.column == column).map(_.profile).foreach { _.histogram.foreach { _.values.foreach { case(key, entry) => 
                println(s"\t $key occured ${entry.absolute} times. ${entry.ratio}")
            }
        }
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getHistogram: (cov: List[Profile], column: String)Unit


In [37]:
getHistogram(Coverages.high, "HCPCS CPT4 TYPE CD 1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

	 CP occured 7260971 times. 0.7003418194858616
	 NullValue occured 1278584 times. 0.12332315401418224
	 HG occured 61140 times. 0.005897131229881731
	 HC occured 1767058 times. 0.17043789527007444


In [38]:
getHistogram(Coverages.full, "SERVICE FROM DATE SKEY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

	 20181113 occured 492486 times. 0.04750171035131721
	 20181121 occured 393083 times. 0.03791400123054629
	 20181102 occured 407271 times. 0.03928247519014004
	 20181103 occured 111164 times. 0.01072209185539046
	 20181114 occured 481536 times. 0.04644555093085261
	 20181104 occured 92891 times. 0.008959607737568594
	 20181115 occured 436640 times. 0.04211520085403269
	 20181129 occured 466324 times. 0.044978309186185284
	 20181108 occured 473367 times. 0.0456576270673115
	 20181119 occured 502723 times. 0.0484890988433077
	 20181107 occured 480598 times. 0.04635507809647857
	 20181118 occured 100351 times. 0.009679146484296066
	 20181120 occured 474406 times. 0.04575784164611175
	 20181125 occured 99409 times. 0.009588287838261579
	 20181126 occured 500263 times. 0.04825182467213484
	 20181110 occured 112710 times. 0.010871208062151944
	 20181124 occured 109190 times. 0.0105316938009615
	 20181106 occured 485789 times. 0.04685576517881936
	 20181130 occured 416907 times. 0.04021189548

In [39]:
getStatistics(Coverages.full, "SERVICE FROM DATE SKEY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**** SERVICE FROM DATE SKEY ****
	**Statistics
	minimum: 2.0181101E7
	maximum: 2.018113E7
	mean: 2.018111541915206E7
	standard deviation: 8.929153507544166



In [40]:
df_claim.select("SOURCE MEMBER ID", "MEMBER KEY").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|SOURCE MEMBER ID| MEMBER KEY|
+----------------+-----------+
|       768718931|35856706062|
|       574453334|36333952331|
|       632180015|38734431603|
|       632180015|38734431603|
|       632180015|38734431603|
|       294376436|35804550111|
|       294376436|35804550111|
|       073715054|35949961362|
|       073715054|35949961362|
|       073715054|35949961362|
|       073715054|35949961362|
|       761139541|17502265693|
|       325696581|35850290440|
|       920761205|35994416047|
|       875939821|36144412508|
|       310944203|35944779958|
|       310944203|35944779958|
|       058015273|35994503060|
|       016104183|35759134125|
|       016104183|35759134125|
+----------------+-----------+
only showing top 20 rows



In [37]:
val df_members = df_claim.groupBy("SOURCE MEMBER ID").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_members: org.apache.spark.sql.DataFrame = [SOURCE MEMBER ID: string, count: bigint]


In [38]:
df_members.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res30: Long = 980184


In [39]:
val df_members2 = df_claim.groupBy("SOURCE MEMBER ID", "MEMBER KEY").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_members2: org.apache.spark.sql.DataFrame = [SOURCE MEMBER ID: string, MEMBER KEY: string ... 1 more field]


In [40]:
df_members2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res31: Long = 980630


In [44]:
val df_sources = df_members2.groupBy("SOURCE MEMBER ID").agg(count("*").alias("keys")).groupBy("keys").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_sources: org.apache.spark.sql.DataFrame = [keys: bigint, count: bigint]


In [45]:
df_sources.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|keys| count|
+----+------+
|   1|979744|
|   3|     6|
|   2|   434|
+----+------+



In [63]:
val df_null_hcpcs = df_claim.filter($"HCPCS CPT4 TYPE CD 1".isNull)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_null_hcpcs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SOURCE PLATFORM CD: string, SOURCE CLAIM NBR: string ... 157 more fields]


In [64]:
val columnNamesString = (s"SOURCE PLATFORM CD|SOURCE CLAIM NBR| SOURCE CLAIM SUFFIX|SOURCE LINE ID|SOURCE LINE SUFFIX|SOURCE MEMBER ID| MEMBER KEY|MEMBER AGE|SOURCE AUTHORIZATION KEY| PLAN GEN KEY|CLAIM FORM TYPE CD"+
s"|SERVICE FROM DATE SKEY|SERVICE TO DATE SKEY|BILL TYPE CD|SOURCE ADMIT DATE SKEY|ADMIT TYPE CD|ADMITTING DIAG CD|CAUSE SUB CD|PRIMARY DIAG CD"+
s"|DIAG LINE POINTER NBR|HCPCS CPT4 BASE CD 1|HCPCS CPT4 SUFFIX CD 1|HCPCS CPT4 TYPE CD 1|CPT MODIFIER CD|CPT MODIFIER CD 2|CPT MODIFIER CD 3|CPT MODIFIER CD 4|NATIONAL DRUG CD ID|"+ 
s"DRG STANDARD CD|IN PLAN NETWORK IND")
val columnNamesList = columnNamesString.split("[|]").map(_.trim)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

columnNamesString: String = SOURCE PLATFORM CD|SOURCE CLAIM NBR| SOURCE CLAIM SUFFIX|SOURCE LINE ID|SOURCE LINE SUFFIX|SOURCE MEMBER ID| MEMBER KEY|MEMBER AGE|SOURCE AUTHORIZATION KEY| PLAN GEN KEY|CLAIM FORM TYPE CD|SERVICE FROM DATE SKEY|SERVICE TO DATE SKEY|BILL TYPE CD|SOURCE ADMIT DATE SKEY|ADMIT TYPE CD|ADMITTING DIAG CD|CAUSE SUB CD|PRIMARY DIAG CD|DIAG LINE POINTER NBR|HCPCS CPT4 BASE CD 1|HCPCS CPT4 SUFFIX CD 1|HCPCS CPT4 TYPE CD 1|CPT MODIFIER CD|CPT MODIFIER CD 2|CPT MODIFIER CD 3|CPT MODIFIER CD 4|NATIONAL DRUG CD ID|DRG STANDARD CD|IN PLAN NETWORK IND
columnNamesList: Array[String] = Array(SOURCE PLATFORM CD, SOURCE CLAIM NBR, SOURCE CLAIM SUFFIX, SOURCE LINE ID, SOURCE LINE SUFFIX, SOURCE MEMBER ID, MEMBER KEY, MEMBER AGE, SOURCE AUTHORIZATION KEY, PLAN GEN KEY, CLAIM FORM TYPE CD, SERVICE FROM DATE SKEY, SERVICE TO DATE SKEY, BILL TYPE CD, SOURCE ADMIT DATE SKEY, ADMIT TYPE CD, ADMITTING DIAG CD, CAUSE SUB CD, PRIMARY DIAG CD, DIAG LINE POINTER NBR, HCPCS CPT4 BASE CD 1,

In [65]:
df_null_hcpcs.select(columnNamesList.map(col): _*).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+--------------------+--------------+------------------+----------------+-----------+----------+------------------------+-------------+------------------+----------------------+--------------------+------------+----------------------+-------------+-----------------+------------+---------------+---------------------+--------------------+----------------------+--------------------+---------------+-----------------+-----------------+-----------------+-------------------+---------------+-------------------+
|SOURCE PLATFORM CD|SOURCE CLAIM NBR| SOURCE CLAIM SUFFIX|SOURCE LINE ID|SOURCE LINE SUFFIX|SOURCE MEMBER ID| MEMBER KEY|MEMBER AGE|SOURCE AUTHORIZATION KEY| PLAN GEN KEY|CLAIM FORM TYPE CD|SERVICE FROM DATE SKEY|SERVICE TO DATE SKEY|BILL TYPE CD|SOURCE ADMIT DATE SKEY|ADMIT TYPE CD|ADMITTING DIAG CD|CAUSE SUB CD|PRIMARY DIAG CD|DIAG LINE POINTER NBR|HCPCS CPT4 BASE CD 1|HCPCS CPT4 SUFFIX CD 1|HCPCS CPT4 TYPE CD 1|CPT MODIFIER CD|CPT MODIFIER CD 2|CPT

In [66]:
df_claim.filter($"SOURCE CLAIM NBR"=== "820183340284616").filter($"HCPCS CPT4 TYPE CD 1".isNotNull).select(columnNamesList.map(col): _*).show(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+-------------------+--------------+------------------+----------------+----------+----------+------------------------+------------+------------------+----------------------+--------------------+------------+----------------------+-------------+-----------------+------------+---------------+---------------------+--------------------+----------------------+--------------------+---------------+-----------------+-----------------+-----------------+-------------------+---------------+-------------------+
|SOURCE PLATFORM CD|SOURCE CLAIM NBR|SOURCE CLAIM SUFFIX|SOURCE LINE ID|SOURCE LINE SUFFIX|SOURCE MEMBER ID|MEMBER KEY|MEMBER AGE|SOURCE AUTHORIZATION KEY|PLAN GEN KEY|CLAIM FORM TYPE CD|SERVICE FROM DATE SKEY|SERVICE TO DATE SKEY|BILL TYPE CD|SOURCE ADMIT DATE SKEY|ADMIT TYPE CD|ADMITTING DIAG CD|CAUSE SUB CD|PRIMARY DIAG CD|DIAG LINE POINTER NBR|HCPCS CPT4 BASE CD 1|HCPCS CPT4 SUFFIX CD 1|HCPCS CPT4 TYPE CD 1|CPT MODIFIER CD|CPT MODIFIER CD 2|CPT MODIF